In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
webExtract = pd.read_csv("data/nyc_inspection_data/WebExtract.txt").fillna(0)

/Users/yutongpang/.virtualenvs/sqlVen27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(webExtract.CAMIS.unique())

25232

In [4]:
webExtract["INSPDATE"] = pd.to_datetime(webExtract.INSPDATE)

In [5]:
webExtract = webExtract.sort_values(by=["INSPDATE","SCORE"])

In [6]:
webExtract = webExtract.groupby("CAMIS").last()

In [7]:
webExtract['CAMIS'] = webExtract.index

In [8]:
webExtract.head()

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE,CAMIS
CAMIS,,,,,,,,,,,,,,,
30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462.0,7188924968,8,2014-03-03,D,10F,2.0,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000,30075445
30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225.0,7182875005,39,2014-07-01,F,04J,23.0,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000,30112340
30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22,D,10F,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000,30191841
40356018,RIVIERA CATERER,3,2780,STILLWELL AVENUE,11224.0,7183723031,3,2014-06-10,D,10F,5.0,A,2014-06-10 00:00:00,2014-09-04 06:01:28.403000000,40356018
40356068,TOV KOSHER KITCHEN,4,97-22,63 ROAD,11374.0,7188967788,50,2014-01-29,P,08A,12.0,0,0,2014-09-04 06:01:28.403000000,40356068


In [11]:
borogroup = webExtract.groupby("BORO").agg({'CAMIS':{"number":lambda x: x.nunique()},"SCORE":{"mean":np.mean,"std":np.std}})

In [12]:
borogroup.head()

SCORE             CAMIS
           std       mean number
BORO                            
0     6.020168   9.666667     12
1     8.068777  10.722184  10201
2     7.394477  10.069767   2365
3     8.605419  10.702404   6072
4     8.332804  11.059733   5625

In [14]:
borogroup.columns = borogroup.columns.droplevel(0)

In [15]:
borogroup['std'] = borogroup['std']/(borogroup.number)**0.5

In [15]:
borogroup = borogroup.sort_values(by="mean")

In [24]:
borodict = {0:"NA",1:"MANHATTAN",2:"THE BRONX",3:"BROOKLYN",4:"QUEENS",5:"STATEN ISLAND"}

In [25]:
boro = map(lambda x1,x2,x3,x4: (borodict[x1],x2,x3,x4) ,borogroup.index, borogroup['mean'], borogroup['std'], borogroup.number)

In [26]:
boro[1:]

[('MANHATTAN', 10.722184099598079, 0.079888882061857444, 10201),
 ('THE BRONX', 10.069767441860465, 0.15205191971516507, 2365),
 ('BROOKLYN', 10.702404479578393, 0.11043485439039237, 6072),
 ('QUEENS', 11.059733333333334, 0.11110405171821128, 5625),
 ('STATEN ISLAND', 10.748171368861025, 0.25165495712645253, 957)]

In [27]:
import pickle

In [29]:
with open('boro.pickle', 'wb') as f:
    pickle.dump(boro[1:], f)